# MHW for 2D-dataset (test)

In [ ]:
# 1. Packages imports (testing_python.py)

import netCDF4 as nc
import numpy as np
from datetime import date
import marineHeatWaves as mhw
import datetime as dt
import xarray as xr
import fsspec
import s3fs
from matplotlib import pyplot as plt
import warnings 
warnings.filterwarnings("ignore") 

In [ ]:
# Instead of netCDF apply the changes for a XArray... 
# Get Megan output 

In [ ]:
# 2. Preparing the data (testing_python.py)

t = np.arange(date(1982,1,1).toordinal(),date(2016,12,31).toordinal()+1)
nc_obj=nc.Dataset("sst_1_2.nc")
sst_1_2=nc_obj.variables['sst'][:]
mhws, clim=mhw.detect(t,sst_1_2,climatologyPeriod=[1982, 2005])

In [ ]:
# 3. More calculation (testing_python.py)

da=nc.Dataset("threshold_climatology","w",format="NETCDF4")
da.createDimension("time",12784)
da.createVariable("threshold","f8",("time"))
da.createVariable("climatology","f8",("time"))
da.variables["threshold"][:]=clim['thresh']
da.variables['climatology'][:]=clim['seas']
da.close()

### Alternatively - a Matlab code to be translated to python
(https://github.com/ZijieZhaoMMHW/m_mhw1.0/blob/master/testing/an_example.m)

In [ ]:
# 1. Import the packages

import netCDF4 as nc
import numpy as np
from datetime import date
import marineHeatWaves as mhw
import datetime as dt
import xarray as xr
import fsspec
import s3fs
from matplotlib import pyplot as plt
import warnings 
warnings.filterwarnings("ignore") 
#what more?

In [ ]:
# 2. Load the (MUR?) dataset

sst_full=NaN(32,32,datenum(2016,12,31)-datenum(1982,1,1)+1); #here it was used NOAA OI SST V2 data
for i=1982:2016
    file_here=['sample_teste_original_code\sst_' num2str(i)];
    load(file_here);
    eval(['data_here=sst_' num2str(i) ';'])
    sst_full(:,:,(datenum(i,1,1):datenum(i,12,31))-datenum(1982,1,1)+1)=data_here;
end

% This data includes SST in [147-155E, 45-37S] in resolution of 0.25 from
% 1982 to 2016.

load('sample_teste_original_code\lon_and_lat');
size(sst_full); %size of data
datenum(2016,12,31)-datenum(1982,1,1)+1; % The temporal length is 35 years.

In [ ]:
# 3. Detecting MHWs and MCSs (copy and paste!)

# Here we detect marine heatwaves off eastern Tasmania based on the traditional definition of MHWs (Hobday et al. 2016). We detected MHWs during 1993 to 2016 for climatologies and thresholds in 1982 to 2005.

[MHW,mclim,m90,mhw_ts]=detect(sst_full,datenum(1982,1,1):datenum(2016,12,31),datenum(1982,1,1),datenum(2005,12,31),datenum(1993,1,1),datenum(2016,12,31)); %take about 30 seconds.

#Additionally, we also detect MCSs during 1982 to 2005 based on the same climatologies. 

[MCS,~,m10,mcs_ts]=detect(sst_full,datenum(1982,1,1):datenum(2016,12,31),datenum(1982,1,1),datenum(2005,12,31),datenum(1993,1,1),datenum(2016,12,31),'Event','MCS','Threshold',0.1);

# Have a look of these two data.
MHW(1:5,:);
MCS(1:5,:);

# You could see that the properties `mhw_onset` and `mhw_end` are in a strange format. This is due to the fact that they are originally constructed in numeric format. We could change it to date format by following steps.

datevec(num2str(MHW{1:5,:}),'yyyymmdd') % vector
datestr(datevec(num2str(MHW{1:5,:}),'yyyymmdd')) % string
datenum(num2str(MHW{1:5,:}),'yyyymmdd') % number

In [ ]:
# 4. Mean states and trends

[mean_freq,annual_freq,trend_freq,p_freq]=mean_and_trend(MHW,mhw_ts,1982,'Metric','Frequency');
metric_used={'Frequency','MeanInt','MaxInt','CumInt','Duration','Days'};

for i=1:6
    eval(['[mean_' metric_used{i} ',annual_' metric_used{i} ',trend_' metric_used{i} ',p_' metric_used{i} ']=mean_and_trend(MHW,mhw_ts,1993,' '''' 'Metric' '''' ',' 'metric_used{i}' ');'])

In [ ]:
# 5. Plot the means and trends - Here it is used m_map... Python alternative?

%matplotlib inline

#m_proj('mercator','long',[146 156],'lat',[-46 -36]);

subplot(2,6,1);
m_contourf(lon_used,lat_used,mean_Frequency',2:0.1:5,'linestyle','none');
m_gshhs_h('patch',[1 1 1]);
m_grid;
colormap(jet);
s=colorbar('location','southoutside');
m_text(148,-44,'a) Frequency','fontweight','bold','fontsize',14);

subplot(2,6,2);
m_contourf(lon_used,lat_used,mean_MeanInt',1:0.1:2.5,'linestyle','none');
m_gshhs_h('patch',[1 1 1]);
m_grid;
colormap(jet);
m_text(148,-44,'b) MeanInt','fontweight','bold','fontsize',14);
s=colorbar('location','southoutside');

subplot(2,6,3);
m_contourf(lon_used,lat_used,mean_MaxInt',1:0.1:3,'linestyle','none');
m_gshhs_h('patch',[1 1 1]);
m_grid;
colormap(jet);
m_text(148,-44,'c) MaxInt','fontweight','bold','fontsize',14);
s=colorbar('location','southoutside');

subplot(2,6,4);
m_contourf(lon_used,lat_used,mean_CumInt',15:0.1:50,'linestyle','none');
m_gshhs_h('patch',[1 1 1]);
m_grid;
colormap(jet);
m_text(148,-44,'d) CumInt','fontweight','bold','fontsize',14);
s=colorbar('location','southoutside');

subplot(2,6,5);
m_contourf(lon_used,lat_used,mean_Duration',9:0.1:27,'linestyle','none');
m_gshhs_h('patch',[1 1 1]);
m_grid;
colormap(jet);
m_text(148,-44,'e) Duration','fontweight','bold','fontsize',14);
s=colorbar('location','southoutside');

subplot(2,6,6);
m_contourf(lon_used,lat_used,mean_Days',26:0.1:72,'linestyle','none');
m_gshhs_h('patch',[1 1 1]);
m_grid;
colormap(jet);
m_text(148,-44,'f) Days','fontweight','bold','fontsize',14);
s=colorbar('location','southoutside');

subplot(2,6,7);
m_contourf(lon_used,lat_used,(trend_Frequency')*10,0.5:0.05:3.5,'linestyle','none');
s=colorbar('location','southoutside');
m_gshhs_h('patch',[1 1 1]);
m_grid;
[lon,lat]=meshgrid(lon_used,lat_used);
lon=lon';
lat=lat';
hold on
m_scatter(lon(p_Frequency<0.05),lat(p_Frequency<0.05),1.5,'k');
colormap(jet);
m_text(148,-44,'g) t-Frequency','fontweight','bold','fontsize',14);

subplot(2,6,8);
m_contourf(lon_used,lat_used,(trend_MeanInt')*10,-0.4:0.05:0.4,'linestyle','none');
m_gshhs_h('patch',[1 1 1]);
m_grid;
s=colorbar('location','southoutside');
[lon,lat]=meshgrid(lon_used,lat_used);
lon=lon';
lat=lat';
hold on
m_scatter(lon(p_MeanInt<0.05),lat(p_MeanInt<0.05),1.5,'k');
colormap(jet);
caxis([-0.4 0.4]);
m_text(148,-44,'h) t-MeanInt','fontweight','bold','fontsize',14);

subplot(2,6,9);
m_contourf(lon_used,lat_used,(trend_MaxInt')*10,-0.4:0.05:0.4,'linestyle','none');
s=colorbar('location','southoutside');
m_gshhs_h('patch',[1 1 1]);
m_grid;
[lon,lat]=meshgrid(lon_used,lat_used);
lon=lon';
lat=lat';
hold on
m_scatter(lon(p_MaxInt<0.05),lat(p_MaxInt<0.05),1.5,'k');
colormap(jet);
caxis([-0.4 0.4]);
m_text(148,-44,'i) t-MaxInt','fontweight','bold','fontsize',14);

subplot(2,6,10);
m_contourf(lon_used,lat_used,(trend_CumInt')*10,-0.4:0.1:110,'linestyle','none');
s=colorbar('location','southoutside');
m_gshhs_h('patch',[1 1 1]);
m_grid;
[lon,lat]=meshgrid(lon_used,lat_used);
lon=lon';
lat=lat';
hold on
m_scatter(lon(p_CumInt<0.05),lat(p_CumInt<0.05),1.5,'k');
colormap(jet);
caxis([0 110]);
m_text(148,-44,'j) t-CumInt','fontweight','bold','fontsize',14);


subplot(2,6,11);
m_contourf(lon_used,lat_used,(trend_Duration')*10,-3:0.1:42,'linestyle','none');
s=colorbar('location','southoutside');
m_gshhs_h('patch',[1 1 1]);
m_grid;
[lon,lat]=meshgrid(lon_used,lat_used);
lon=lon';
lat=lat';
hold on
m_scatter(lon(p_Duration<0.05),lat(p_Duration<0.05),1.5,'k');
colormap(jet);
caxis([0 40]);
m_text(148,-44,'k) t-Duration','fontweight','bold','fontsize',14);

subplot(2,6,12);
m_contourf(lon_used,lat_used,(trend_Days')*10,0:0.1:75,'linestyle','none');
s=colorbar('location','southoutside');
m_gshhs_h('patch',[1 1 1]);
m_grid;
[lon,lat]=meshgrid(lon_used,lat_used);
lon=lon';
lat=lat';
hold on
m_scatter(lon(p_Days<0.05),lat(p_Days<0.05),1.5,'k');
colormap(jet);
caxis([0 74]);
m_text(148,-44,'l) t-Days','fontweight','bold','fontsize',14);